In [0]:
#imports:
import pandas as pd
import numpy as np
import math
import os
from time import time

from sklearn.utils import shuffle

import xgboost

from scipy.stats import randint as sp_randint # int distribution for random search
from scipy.stats import uniform # float distribution for random search
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from matplotlib import pyplot as plt

In [2]:
from google.colab import drive 
drive.mount('/mntDrive') 


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /mntDrive


In [0]:
PREP_DATA_PATH = "/mntDrive/My Drive/GrBoost/Практика/data/input/prepared_facebook_data.csv"
data=pd.read_csv(PREP_DATA_PATH)

RES_DATA_DIR = "/mntDrive/My Drive/GrBoost/Практика/data/results/test 1"
RES_SAVE_DIR = "/mntDrive/My Drive/GrBoost/Практика/data/results/test 2"

In [4]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


In [0]:
files = os.listdir(RES_DATA_DIR)
files = [x for x in files if x.find("group_search")!=-1]

In [0]:
def group_best_params_with_stop(df, group_num, stop_iter_num, group_size = 243):
    best_params = {}
    stop_iter_nums = {}
    best_score = {}

    tmp_df = df.loc[(df['iter_num'] > (group_num-1)*group_size) & (df['iter_num'] <= group_num*group_size)].copy() #take group
    for i in tmp_df.run_number.unique(): 
      tmp_df_i = tmp_df.loc[tmp_df.run_number==i] #take run
      #cut iterations:
      tmp_df_i_copy = tmp_df_i.copy()
      tmp_df_i_copy.loc[:, "mean_test_score"] = tmp_df_i.loc[:, "mean_test_score"].cummax()      

      try:
          curr_max=tmp_df_i_copy.groupby("mean_test_score").count().reset_index().sort_values(by = "iter_num")
          max_score = curr_max.loc[curr_max["mean_test_score"]>=stop_iter_num].index[0] #get first interval with iter_amount more than stop_iter_num
      except:
          max_score = tmp_df_i_copy.mean_test_score.max()
      
      tmp_stop = tmp_df_i.loc[tmp_df_i["mean_test_score"]==max_score].sort_values(by="iter_num").iloc[0]#.index #get first element with best_score

      best_params[i] = tmp_stop['params']
      #best_score[i] = tmp_stop['mean_test_score']

      #last_iter_num = tmp_stop['iter_num'] + stop_iter_num
      #if last_iter_num > group_size:
      #    last_iter_num = group_size

      #stop_iter_nums[i] = last_iter_num
    return best_params#, stop_iter_nums, best_score       
   

In [0]:
group_df = pd.DataFrame([])
for file_res in files:
    df = pd.read_csv(RES_DATA_DIR+"/"+file_res)    
    df['iter_num'] = range(1, df.shape[0]+1)
    group_df = pd.concat([group_df, df], sort = False)

In [0]:
# get some data
X, y = data.iloc[:,:-1].values, data.iloc[:,-1].values

param_dist_group_2 = {"alpha": uniform(loc=0, scale=1),
                  "lambda": uniform(loc=0, scale=1)}

param_dist_group_3 = {"subsample":uniform(loc=0.5, scale=0.4),
                  "colsample_bytree":uniform(loc=0.5, scale=0.4)}

In [0]:
import json 

In [0]:
stop_iter_num = 50
n_iter_search = 243

best_params_first_group = group_best_params_with_stop(group_df, group_num = 1, stop_iter_num = stop_iter_num)

for params_set_num in best_params_first_group:
  clf = xgboost.XGBRegressor(tree_method = "gpu_hist", gpu_id=0, verbosity=0)
  clf.set_params(**json.loads(best_params_first_group[params_set_num].replace("'", "\"")))

  #second group:
  random_search = RandomizedSearchCV(clf, param_distributions=param_dist_group_2,
                                      n_iter=n_iter_search, cv=5, scoring="r2")

  random_search.fit(X, y)

  res_rand_2=pd.DataFrame(random_search.cv_results_)
  res_rand_2['iter_num'] = range(n_iter_search+1, 2*n_iter_search+1)
  res_rand_2['experiment_name']='random search for 2 group' 
  res_rand_2['run_number']=params_set_num

  #fit last suitable params
  best_params_second_group = group_best_params_with_stop(res_rand_2, group_num = 2, stop_iter_num = stop_iter_num)
  clf.set_params(**best_params_second_group[params_set_num])
  print('done for second group')

  #third group:
  random_search = RandomizedSearchCV(clf, param_distributions=param_dist_group_3,
                                        n_iter=n_iter_search, cv=5, scoring="r2")
  random_search.fit(X, y)

  res_rand_3=pd.DataFrame(random_search.cv_results_)
  res_rand_3['iter_num'] = range(2*n_iter_search+1, 3*n_iter_search+1)
  res_rand_3['experiment_name']='random search for 3 group' 
  res_rand_3['run_number']=params_set_num

  res=pd.concat([res_rand_2,res_rand_3], sort=False)
  res.to_csv(RES_SAVE_DIR+'/GR_S_2_and_3_groups_with_'+str(stop_iter_num)+'_'+str(params_set_num), index=False)
  print('done for third group')